In [ ]:
# This notebook can be used to produce unfolded results, to be used as input for UnfoldingPlot.ipynb.
# Prerequisites: Full set of numpy toys, generated by ToysROOTtoNumpy.ipynb.

In [ ]:
import sys
import numpy as np
import matplotlib.pyplot as plt
from ipynb.fs.full.CoefficientsCalcPlus import GetCoefficientsFlux
from ipynb.fs.full.CoefficientsCalcPlus import get_normalization
import sklearn.linear_model
import sklearn.linear_model as linear_model
import scipy
import warnings; warnings.simplefilter('ignore')
import os
from matplotlib.pyplot import figure
import random as rnd
import matplotlib.ticker as ticker
#import cvxpy as cp
import uproot4 as uproot

def rebin(arr, factor):
    new_size = arr.size // factor
    remainder = arr.size % factor
    
    if remainder != 0:
        arr = arr[:-remainder]
        
    return np.sum(arr.reshape(-1, factor), axis=1)

def rebinned_bin_centers(nbins, xmin, xmax, factor):
    # Compute the bin widths for the original histogram
    bin_width = (xmax - xmin) / nbins
    
    # Compute the bin edges for the original histogram
    edges = np.linspace(xmin, xmax, nbins + 1)
    
    # Compute the new bin widths after rebinning
    new_bin_width = bin_width * factor
    
    # Compute the number of new bins
    new_nbins = int(nbins / factor)
    
    # Compute the bin edges for the rebinned histogram
    new_edges = np.linspace(xmin, xmax, new_nbins + 1)
    
    # Compute the bin centers for the rebinned histogram
    centers = (new_edges[1:] + new_edges[:-1]) / 2
    
    return centers

def shift(toy,energy):
    toy = np.roll(toy,energy,axis=-1)
    toy[..., -energy:] = 0
    return toy

#function definitions
def chi2sum(reco,folded,recoerror):
    res = ((np.array(reco)-np.array(folded))/np.array(recoerror))**2
    return res[res != np.inf].sum()

def c_norm(trueRidge):
    return np.sqrt((trueRidge**2).sum())

def GetGaussianRow(energy_bin_centers,target_loc,target_scale,rescaling = 1):
    return np.array([scipy.stats.norm.pdf(x, loc = target_loc, scale = target_scale) for x in energy_bin_centers])

def integrate(unfolded,rebin_factor):
    return np.array(unfolded).sum()*rebin_factor*0.001

def curvature(x_ridge,y_ridge,alphas):
    fac = 1.

    res = x_ridge
    sol = y_ridge
    dl = np.diff(alphas)
    xi = np.log(sol)
    rho = np.log(res)
    xi_prime = (1/fac)*np.diff(xi)/dl
    rho_prime = np.diff(rho)/dl
    xi_prime_prime = fac*np.diff(xi_prime)/dl[:-1]
    rho_prime_prime = np.diff(rho_prime)/dl[:-1]

    curv = 2*(rho_prime[:-1]*xi_prime_prime - rho_prime_prime*xi_prime[:-1])/np.power(np.power(rho_prime[:-1], 2) + np.power(xi_prime[:-1], 2), 3./2)

    curvature = plt.plot(alphas[1:-1], curv, "r", label = "Curvature")
    plt.xlabel("Regularisation strength")
    plt.ylabel("L-curve Curvature")
    plt.xscale("log")
    plt.legend()
    plt.show(curvature)
    
    maxCurv = np.max(curv[~np.isnan(curv)])
    num = alphas[1:-1][curv == maxCurv][0]
    print(f'Optimal regularisation strength = {num}')
    return num

def unfold(smearing,reco,alpha_val,modified=False):
    if modified:
        print("modified")
        unfolded = cp.Variable(len(reco))
        obj = cp.Minimize(cp.sum_squares(smearing @ unfolded - reco) + alpha_val * cp.norm(unfolded,2.5))
        #obj = cp.Minimize(cp.sum_squares(smearing @ unfolded - reco) + alpha_val * cp.tv(unfolded))
        prob = cp.Problem(obj)
        prob.solve()
        unfolded = unfolded.value
        return unfolded
    else:
        clf = sklearn.linear_model.Ridge(alpha=alpha_val,fit_intercept=False)
        clf.fit(smearing,reco)
        res = clf.coef_
        return np.array(res)

In [ ]:
def FlexUnfold(rebin_factor,energy,energy_width,input_flux_alpha=-1,smearing_flag=True):
    if input_flux_alpha == -1:
        flux_alpha_flag = False
    else:
        flux_alpha_flag = True
    flux_alpha_flag = False if input_flux_alpha == -1 else True
    
    flux_alpha = input_flux_alpha if flux_alpha_flag else 1e-12
    
    ## Calling this creates a 900 x 58 x (bins) numpy array containing 900 toys ready for building a virtual flux with center = energy.
    coeffs, std = GetCoefficientsFlux(1e-3*energy,1e-3*energy_width,flux_alpha,model=sklearn.linear_model.Ridge,years=1)
    coeffs = np.array(coeffs)
    normalization = get_normalization(coeffs)
    energy_bin_centers = rebinned_bin_centers(16000,-8000,8000,rebin_factor)/1000
    
    ## Get toys
    reco_toys = []
    for i in range(733):
        filename = f'FixedUncNumpyELep/ELepToy{i}.npy'
        toy = np.load(filename)
        toy = shift(toy,energy)
        toy = np.apply_along_axis(rebin,1,toy,rebin_factor)
        reco_toys.append(toy)

    true_toys = []
    for i in range(733):
        filename = f'FixedUncNumpyTrueOmega/TrueToy{i}.npy'
        toy = np.load(filename)[:,:8000]
        pad_width = [(0, 0), (8000, 0)]  # pad 8000 cells of 0 at the start of each row
        toy = np.pad(toy, pad_width, mode='constant')
        toy = np.apply_along_axis(rebin,1,toy,rebin_factor)
        true_toys.append(toy)
    
    print("Done creating correctly binned toys")
    
    reco_toys_array = np.array(reco_toys)
    true_toys_array = np.array(true_toys)
    
    reco_array = [np.sum(reco_toys_array[i] * coeffs[:, np.newaxis], axis=0) for i in range(len(reco_toys_array))]
    reco_array = np.array(reco_array)

    true_array = [np.sum(true_toys_array[i] * coeffs[:, np.newaxis], axis=0) for i in range(len(true_toys_array))]
    true_array = np.array(true_array)
    
    E = (12/201)*1.1e21
    nucleons = 1.3954*(2*3*0.574)*1e3/1.66e-27
    ftilde = 1e38*(1/(E*nucleons*get_normalization(coeffs)))/(1e-3*rebin_factor)
    
    reco = ftilde*reco_array.mean(axis=0)
    true = ftilde*true_array.mean(axis=0)
    
    loc = energy
    smearing = virtual_smearing(energy,width,1e-12,rebin_factor) if smearing_flag else np.array([GetGaussianRow(energy_bin_centers,loc,std)*0.001*rebin_factor for loc in energy_bin_centers])[:,16000//(2*rebin_factor):]
    
    alphas = np.logspace(-4,4,1000) ## Sets the range of alpha that will define the L-curve
    chi2_array = []
    c_norm_array = []
    integral_array = []

    modified = False # Default

    if modified
        # This part of the code can be used to explore custom regularization schemes,
        # For example, different power law for ||c|| (for Tikhonov it's 2) or others (such as Total Variation)
        alphas = np.logspace(-2,0,200) ## fits L2.5
        ## alphas = np.logspace(-4,0,200) ## fits TV
        for alpha_val in alphas:
            unfolded = cp.Variable(len(reco))

            obj = cp.Minimize(cp.sum_squares(smearing @ unfolded - reco) + alpha_val * cp.norm(unfolded,2.5))
            #obj = cp.Minimize(cp.sum_squares(smearing @ unfolded - reco) + alpha_val * cp.tv(unfolded))
            prob = cp.Problem(obj)
            prob.solve()

            unfolded = unfolded.value

            integral = integrate(unfolded,rebin_factor)
            folded = np.matmul(smearing,unfolded)
            chi2_array.append(chi2sum(reco,folded,recoerror))
            c_norm_array.append(c_norm(unfolded))
            integral_array.append(-integral)
    else:
        for alpha_val in alphas:
            clf = sklearn.linear_model.Ridge(alpha=alpha_val,fit_intercept=False)
            clf.fit(smearing,reco)
            trueRidge = clf.coef_
            integral = integrate(trueRidge,rebin_factor)
            folded = np.matmul(smearing,trueRidge)
            #chi2_array.append(chi2sum(reco,folded,recoerror))
            c_norm_array.append(c_norm(trueRidge))
            integral_array.append(-integral)

    optimal = curvature(c_norm_array,-1*np.array(integral_array),alphas)
    print("optimal = "+str(optimal))
    
    print("Done calculating optimal regularization parameter")
    
    rand_recos_stat = []
    for i in range(1000):
        filename = f'NumpyELepStat/ELepStatToy{i}.npy' ## These are toys based on 20 years of data
        toy = np.load(filename)
        toy = shift(toy,energy)
        toy = np.apply_along_axis(rebin,1,toy,rebin_factor)
        years = 20
        virtual_flux = toy.transpose().dot(np.array(coeffs))
        E = (12/201)*1.1e21
        nucleons = 1.3954*(2*3*0.574)*1e3/1.66e-27
        ftilde = 1e38*(1/(E*nucleons*normalization))/(1e-3*rebin_factor)
        rand_recos_stat.append(ftilde*virtual_flux/years)
    rand_recos_stat = np.array(rand_recos_stat)
    
    rand_recos_sys = ftilde*reco_array
    
    print("Recos stat length = "+str(rand_recos_stat.shape))
    print("Recos sys length = "+str(rand_recos_sys.shape))
    
    print("Done getting stat and sys toys")
    
    solutions_sys = np.array([unfold(smearing,rand_reco,optimal,modified=modified) for rand_reco in rand_recos_sys]).transpose()
    solutions_stat = np.array([unfold(smearing,rand_reco,optimal,modified=modified) for rand_reco in rand_recos_stat]).transpose()
    
    smearing_string = "GaussianSmearing"
    np.save("FixedUncFlexibleUnfoldingResults/"+smearing_string+"SystematicUnfoldingEnergy"+str(energy)+"BinWidth"+str(rebin_factor)+"FluxWidth"+str(energy_width)+"Alpha"+str(flux_alpha)+".npy",solutions_sys)
    np.save("FixedUncFlexibleUnfoldingResults/"+smearing_string+"StatisticalUnfoldingEnergy"+str(energy)+"BinWidth"+str(rebin_factor)+"FluxWidth"+str(energy_width)+"Alpha"+str(flux_alpha)+".npy",solutions_stat)
    np.save("FixedUncFlexibleUnfoldingResults/"+smearing_string+"TrueEnergy"+str(energy)+"BinWidth"+str(rebin_factor)+"FluxWidth"+str(energy_width)+"Alpha"+str(flux_alpha)+".npy",true)
    
    print("Done!")

In [ ]:
energy = 500
rebin_factor = 40
width = 70
years = 20
flux_alpha = 1e-12

modes = ['CCQE','RES','2p2h','Other']
modes_arrays = [np.load("Mode"+modes[i]+"Numpy_20Years.npy") for i in range(len(modes))] #Generated in ToysROOTtoNumpy.ipynb

for energy in [500,625,750,875,1000]: #[500,625,750,875,1000]
    for rebin_factor in [40,60,80,100]: #[40,60,80,100]
        for width in [70,100,130]: #[70,100,130]
            print('('+str(energy)+','+str(width)+','+str(rebin_factor)+')')
            FlexUnfold(rebin_factor,energy,width,input_flux_alpha=1e-12,smearing_flag=False)
            for i in range(len(modes)):
                ## This is for creating mode hists
                coeffs, std = GetCoefficientsFlux(1e-3*energy,1e-3*width,flux_alpha,model=sklearn.linear_model.Ridge,years=1)
                coeffs = np.array(coeffs)
                normalization = get_normalization(coeffs)

                E = (12/201)*1.1e21
                nucleons = 1.3954*(2*3*0.574)*1e3/1.66e-27
                ftilde = 1e38*(1/(E*nucleons*get_normalization(coeffs)))/(1e-3*rebin_factor*years)

                true_res = rebin(ftilde*np.sum(modes_arrays[i] * coeffs[:, np.newaxis], axis=0),rebin_factor)
                np.save("UnfoldingResults/GaussianSmearingTrueEnergy"+str(energy)+"BinWidth"+str(rebin_factor)+"FluxWidth"+str(width)+"Mode"+modes[i]+"Alpha"+str(flux_alpha)+".npy",true_res)